In [189]:
import math
import pandas as pd
import copy
import matplotlib.pyplot as plt
from prettytable import PrettyTable
from prettytable import MARKDOWN
from numpy.random import normal
import numpy as np
import matplotlib

In [190]:
file = "user1.txt"
df = pd.read_csv(file, sep=',')



In [191]:
df.head()

,Unnamed: 0,lat,lng,datetime,elevation,pace_metric
0,0,59.316655,18.050860,2024-03-31 00:00:37+00:00,21.0,6.038254
1,1,59.316546,18.050918,2024-03-31 00:00:42+00:00,21.0,6.622239
2,2,59.316422,18.050833,2024-03-31 00:00:47+00:00,21.0,5.693252
3,3,59.316269,18.050892,2024-03-31 00:00:54+00:00,21.0,6.715673
4,4,59.316202,18.050867,2024-03-31 00:00:56+00:00,21.0,4.386885
5,5,59.315165,18.051376,2024-03-31 00:01:39+00:00,19.0,6.017173
6,6,59.315091,18.051467,2024-03-31 00:01:43+00:00,19.0,6.846535
7,7,59.314918,18.051618,2024-03-31 00:01:50+00:00,19.0,5.528229
8,8,59.314849,18.051655,2024-03-31 00:01:53+00:00,18.0,6.273351
9,9,59.314849,18.051655,2024-03-31 00:01:53+00:00,18.0,6.273351


In [192]:
df.sort_values(by=["datetime"], inplace=True)

In [193]:
# Here, we get the unique trajectories
unique_uids = df.drop_duplicates(["datetime"])
unique_uids = unique_uids["datetime"].to_list()


In [194]:
class Point:
    R = 6371 # Radius of the earth
    def __init__(self, latitude, longitude, t):
        self.latitude = latitude
        self.longitude = longitude
        self.t = t

    def __str__(self):
        return "Point({},{},{})".format(self.latitude, self.longitude, self.t)

    def getDistance(self, point2):
        delta_lambda = math.radians(point2.latitude - self.latitude)
        delta_phi = math.radians(point2.longitude - self.longitude)
        a = math.sin(delta_lambda / 2) * math.sin(delta_lambda / 2) + math.cos(math.radians(self.latitude)) \
            * math.cos(math.radians(point2.latitude)) * math.sin(delta_phi / 2) * math.sin(delta_phi / 2)
        c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
        distance = Point.R * c
        return distance

In [195]:
class Trajectory:
    def __init__(self, points = []):
        self.points = points

    def addNoise(self, mu, sigma):
        return Trajectory([Point(i.latitude + normal(mu, sigma), i.longitude + normal(mu, sigma), i.t) for i in self.points])


In [196]:
# Then, we split the trajectories
points = []
trajectories = []
for i in range(len(unique_uids)):
    #for i in range(1):
    tmp_df = df.loc[df["datetime"]==unique_uids[i]]
    len(tmp_df)
    for j in range(len(tmp_df)):
        points.append(Point(tmp_df.iloc[j][0], tmp_df.iloc[j][1], tmp_df.iloc[j][2]))
    trajectories.append(copy.deepcopy(points))
    points.clear()


/tmp/ipykernel_76175/2748556928.py:9: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  points.append(Point(tmp_df.iloc[j][0], tmp_df.iloc[j][1], tmp_df.iloc[j][2]))


In [197]:
len(trajectories)

69

In [198]:
P = Trajectory(trajectories[0])

In [204]:
import skmob
tdf = skmob.TrajDataFrame.from_file('user1.txt', datetime='time', latitude='lat', longitude='lon')
# print a portion of the TrajDataFrame
print(tdf.head())



#Q = P.addNoise(0, 0.0005)

   Unnamed: 0        lat        lng                  datetime  elevation  \
0           0  59.316655  18.050860 2024-03-31 00:00:37+00:00       21.0   
1           1  59.316546  18.050918 2024-03-31 00:00:42+00:00       21.0   
2           2  59.316422  18.050833 2024-03-31 00:00:47+00:00       21.0   
3           3  59.316269  18.050892 2024-03-31 00:00:54+00:00       21.0   
4           4  59.316202  18.050867 2024-03-31 00:00:56+00:00       21.0   

   pace_metric  
0     6.038254  
1     6.622239  
2     5.693252  
3     6.715673  
4     4.386885  


In [205]:
# Trying to grip the syntax.
tdf.plot_trajectory(zoom=12, weight=3, opacity=0.9, max_points=None, max_users=3)

In [201]:
tdf.to_csv("26kLongRun.txt")